![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F04+-+scikit-learn&dt=04g+-+Vertex+AI+Custom+Model+-+scikit-learn+-+Hyperparameter+Tuning+Job+With+Python+file.ipynb)

# 04g - Vertex AI > Training > Hyperparameter Tuning Jobs

**04 Series Overview**

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) notebook, the model training happened directly in the notebook.  The model was then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `04a-04i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI Training as managed jobs.  These jobs will be demonstrated as:

-  Custom Job that trains and saves (to GCS) a model from a python script (`04a`), python source distribution (`04b`), and custom container (`04c`)
-  Training Pipeline that trains and registers a model from a python script (`04d`), python source distribution (`04e`), and custom container (`04f`)
-  Hyperparameter Tuning Jobs from a python script (`04g`), python source distribution (`04h`), and custom container (`04i`)

**This Notebook (`04g`): An extension of `04a` with Hyperparameter Tuning - And Tensorboard HParams**

This notebook trains the same scikit-learn Keras model from [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a Python script as shown in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Script.ipynb). 

The script is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service. 

The Custom Job is then used as the input for a Vertex AI > Training > Hyperparameter Tuning Job.  This runs and manages the tuning loops for the number of trials in each loop, collects the metric(s) and manages the parameters with the selected search algorithm for parameter modification.

This job is launched using the Vertex AI client library:
- [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference)
    - [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)
        - [`aiplatform` package](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
            - Custom Job:
                - [`aiplatform.CustomJob.from_local_script()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob_from_local_script)
            - Hyperparameter Tuning Job:
                - [`aiplatform.HyperparameterTuningJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.HyperparameterTuningJob#google_cloud_aiplatform_HyperparameterTuningJob)

**Vertex AI Training**

In Vertex AI Training you can run your training code as a job where you specify the compute resources to use. For tips on preparing code, running training jobs, and workflows for building custom containers with software and training code combined please visit these [tips notebooks](../Tips/readme.md) in this repository:
- [Python Packages](../Tips/Python%20Packages.ipynb)
- [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb)
- [Python Training](../Tips/Python%20Training.ipynb)

<p align="center" width="100%">
    <img src="../architectures/overview/training.png" width="45%">
    &nbsp; &nbsp; &nbsp; &nbsp;
    <img src="../architectures/overview/training2.png" width="45%">
</p>

**Prerequisites:**
-  [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)
-  Understanding:
    -  Model overview in [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb)
    -  Convert notebook code to Python Script in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Script.ipynb)

**Resources:**
-  [sklearn](https://scikit-learn.org/stable/)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)
-  Vertex AI Hyperparameter Tuning
   -  [Overview of Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)
   -  [Using Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05g_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05g_console.png" width="45%">
</p>

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [12]:
REGION = 'us-central1'
EXPERIMENT = '04g'
SERIES = '04'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id-splits' + '-' + VAR_TARGET # add more variables to the string with space delimiters
SOLVER = 'newton-cg'
PENALTY = 'l2'


packages:

In [13]:
from google.cloud import aiplatform
from datetime import datetime
import pkg_resources
from IPython.display import Markdown as md
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd

clients:

In [14]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)

parameters:

In [15]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"
BLOB = f"{SERIES}/{EXPERIMENT}/models/{TIMESTAMP}/model/model.pkl"

In [16]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'mg-ce-demos-main@mg-ce-demos.iam.gserviceaccount.com'

List the service accounts current roles:

In [17]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/aiplatform.admin
roles/bigquery.admin
roles/editor
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [18]:
!rm -rf {DIR}
!mkdir -p {DIR}

Experiment Tracking:

In [19]:
FRAMEWORK = 'sklearn'
TASK = 'classification'
MODEL_TYPE = 'logistric-regression'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [20]:
aiplatform.init(experiment = EXPERIMENT_NAME)

---
## Training

### Python File for Training

This notebook trains the same scikit-learn Keras model from [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a python script as shown in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Hyperparameter Tuning Script.ipynb](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Hyperparameter%20Tuning%20Script.ipynb) which stores the script in [`./code/hp_train.py`](./code/hp_train.py).

**Review the script:**

In [21]:
SCRIPT_PATH = './code/hp_train.py'

with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python


# package import
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

import pickle
import pandas as pd 
import numpy as np 

from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud import storage
import argparse
import os
import sys
import hypertune
#from google.cloud.aiplatform import hyperparameter_tuning as hpt

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--penalty', dest = 'penalty', default = 'l2', type = str, help = 'Penalty term')
parser.add_argument('--solver', dest = 'solver', default = 'newton-cg', type = str, help = 'Logistic regression solver')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str)
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--bq_project', dest = 'bq_project', type=str)
parser.add_argument('--bq_dataset', dest = 'bq_dataset', type=str)
parser.add_argument('--bq_table', dest = 'bq_table', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest = 'experiment', type=str)
parser.add_argument('--series', dest = 'series', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)
parser.add_argument('--bucket', dest = 'bucket', type=str)
parser.add_argument('--uri', dest = 'uri', type=str)
parser.add_argument('--blob', dest = 'blob', type=str)
parser.add_argument('--timestamp', dest = 'timestamp', type=str)

# hyperparameters
parser.add_argument('--C', dest = 'C', type=str)
args = parser.parse_args()

# Model Training
VAR_TARGET = str(args.var_target)
VAR_OMIT = str(args.var_omit).split('-')

# clients
bq = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)
hpt = hypertune.HyperTune()
args.run_name = f'{args.run_name}-{hpt.trial_id}'

# Vertex AI Experiment
if args.run_name in [run.name for run in aiplatform.ExperimentRun.list(experiment = args.experiment_name)]:
    expRun = aiplatform.ExperimentRun(run_name = args.run_name, experiment = args.experiment_name)
else:
    expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment': args.experiment, 'series': args.series, 'project_id': args.project_id})
expRun.log_params({'hyperparameter.C': args.C})

# get schema from bigquery source
query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{args.bq_table}'"
schema = bq.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bq.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE {VAR_TARGET} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}', 'nclasses': nclasses, 'var_split': 'splits', 'var_target': VAR_TARGET})

train_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'TRAIN'"
train = bq.query(train_query).to_dataframe()
X_train = train.loc[:, ~train.columns.isin(VAR_OMIT)]
y_train = train[VAR_TARGET].astype('int')

val_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'VALIDATE'"
val = bq.query(val_query).to_dataframe()
X_val = val.loc[:, ~val.columns.isin(VAR_OMIT)]
y_val = val[VAR_TARGET].astype('int')

test_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'TEST'"
test = bq.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(VAR_OMIT)]
y_test = test[VAR_TARGET].astype('int')

# Logistic Regression
# instantiate the model 
logistic = LogisticRegression(solver=args.solver, penalty=args.penalty, C=args.C)

# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

expRun.log_params({'solver': args.solver, 'penalty': args.penalty, 'C':args.C})

# define pipeline
pipe = Pipeline(steps=[("scaler", scaler), ("logistic", logistic)])

# define grid search model
model = pipe.fit(X_train, y_train)

# test evaluations:
y_pred = model.predict(X_test)
test_acc = metrics.accuracy_score(y_test, y_pred) 
test_prec = metrics.precision_score(y_test, y_pred)
test_rec = metrics.recall_score(y_test, y_pred)
test_rocauc = metrics.roc_auc_score(y_test, y_pred)
expRun.log_metrics({'test_accuracy': test_acc, 'test_precision': test_prec, 'test_recall': test_rec, 'test_roc_auc': test_rocauc})

# val evaluations:
y_pred_val = model.predict(X_val)
val_acc = metrics.accuracy_score(y_val, y_pred_val) 
val_prec = metrics.precision_score(y_val, y_pred_val)
val_rec = metrics.recall_score(y_val, y_pred_val)
val_rocauc = metrics.roc_auc_score(y_val, y_pred_val)
expRun.log_metrics({'validation_accuracy': val_acc, 'validation_precision': val_prec, 'validation_recall': val_rec, 'validation_roc_auc': val_rocauc})

# training evaluations:
y_pred_training = model.predict(X_train)
training_acc = metrics.accuracy_score(y_train, y_pred_training) 
training_prec = metrics.precision_score(y_train, y_pred_training)
training_rec = metrics.recall_score(y_train, y_pred_training)
training_rocauc = metrics.roc_auc_score(y_train, y_pred_training)
expRun.log_metrics({'training_accuracy': training_acc, 'training_precision':training_prec, 'training_recall': training_rec, 'training_roc_auc': training_rocauc})

# report hypertune info back to Vertex AI Training > Hyperparamter Tuning Job
hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag = 'roc_auc',
    metric_value = test_rocauc)

# output the model save files
with open('model.pkl','wb') as f:
    pickle.dump(model,f)
    
# Upload the model to GCS
bucket = storage.Client().bucket(args.bucket)
blob = bucket.blob(args.blob)
blob.upload_from_filename('model.pkl')

expRun.log_params({'model.save': f'{args.uri}/models/{args.timestamp}/model'})
expRun.end_run()

```

### Setup Training Job

In [22]:
CMDARGS = [
    "--penalty=" + PENALTY,
    "--solver=" + SOLVER,
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME,
    "--bucket=" + BUCKET,
    "--uri=" + URI,
    "--blob=" + BLOB,
    "--timestamp=" + TIMESTAMP
]

In [23]:
customJob = aiplatform.CustomJob.from_local_script(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    script_path = SCRIPT_PATH,
    container_uri = TRAIN_IMAGE,
    args = CMDARGS,
    requirements = ['google-cloud-aiplatform', 'protobuf',  'db-dtypes>=1.0.0', 'google-auth>=2.6.0', 'google-cloud-bigquery>=3.0.1'],
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    base_output_dir = f"{URI}/models/{TIMESTAMP}",
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

Training script copied to:
gs://mg-ce-demos/04/04g/models/20230221091933/aiplatform-2023-02-21-09:19:47.840-aiplatform_custom_trainer_script-0.1.tar.gz.


### Setup Hyperparameter Tuning Job

In [27]:
METRIC_SPEC = {
    "auc_roc": "maximize"
}

PARAMETER_SPEC = {
    "C": aiplatform.hyperparameter_tuning.DiscreteParameterSpec(values=[0.1, 1.0, 10.0], scale='linear')
}


In [28]:
tuningJob = aiplatform.HyperparameterTuningJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    custom_job = customJob,
    metric_spec = METRIC_SPEC,
    parameter_spec = PARAMETER_SPEC,
    max_trial_count = 10,
    parallel_trial_count = 3,
    search_algorithm = None,
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

### Run Training Job

In [29]:
tuningJob.run(
    service_account = SERVICE_ACCOUNT
)

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/633472233130/locations/us-central1/hyperparameterTuningJobs/2905748235639324672
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/633472233130/locations/us-central1/hyperparameterTuningJobs/2905748235639324672')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2905748235639324672?project=633472233130
HyperparameterTuningJob projects/633472233130/locations/us-central1/hyperparameterTuningJobs/2905748235639324672 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/633472233130/locations/us-central1/hyperparameterTuningJobs/2905748235639324672 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/633472233130/locations/us-central1/hyperparameterTuningJobs/2905748235639324672 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJ

RuntimeError: Job failed with:
code: 3
message: "Hyperparameter Tuning Trial #1 Failed before any other successful trials were completed. The failed trial had parameters: C=1.0000000000000002, .  The trial\'s error message was: The replica workerpool0-0 exited with a non-zero status of 1. \nTraceback (most recent call last):\n  File \"/usr/lib/python3.7/runpy.py\", line 193, in _run_module_as_main\n    \"__main__\", mod_spec)\n  File \"/usr/lib/python3.7/runpy.py\", line 85, in _run_code\n    exec(code, run_globals)\n  File \"/root/.local/lib/python3.7/site-packages/aiplatform_custom_trainer_script/task.py\", line 102, in <module>\n    model = pipe.fit(X_train, y_train)\n  File \"/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py\", line 335, in fit\n    self._final_estimator.fit(Xt, y, **fit_params_last_step)\n  File \"/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py\", line 1308, in fit\n    % self.C)\nValueError: Penalty term must be positive; got (C=

In [ ]:
tuningJob.resource_name, tuningJob.display_name

Create hyperlinks to job and tensorboard here:

In [ ]:
job_link = f"https://console.cloud.google.com/ai/platform/locations/{REGION}/training/{tuningJob.resource_name.split('/')[-1]}?project={PROJECT_ID}"

print(f'Review the Job here:\n{job_link}')

### Get Best Run

In [ ]:
# if trial.state.name == 'SUCCEEDED'
auc_roc = [trial.final_measurement.metrics[0].value if trial.state.name == 'SUCCEEDED' else 1 for trial in tuningJob.trials]
auc_roc

In [ ]:
best = tuningJob.trials[auc_roc.index(max(auc_roc))]
best

In [ ]:
best.id

---
## Serving

### Upload The Model
Only upload the best model from the hyperparmeter tuning job:

In [ ]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if f'{RUN_NAME}-{best.id}' in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  parent_model,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/{best.id}/model",
        is_default_version = True,
        version_aliases = [f'{RUN_NAME}-{best.id}'],
        version_description = f'{RUN_NAME}-{best.id}',
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}-{best.id}'}        
    )

>**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`
>
>**Retrieve a Model Resource**
>[aiplatform.Model()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
>```Python
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

In [ ]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')

### Vertex AI Experiment Update and Review

In [ ]:
expRun = aiplatform.ExperimentRun(run_name = f'{RUN_NAME}-{best.id}', experiment = EXPERIMENT_NAME)

In [ ]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'hyperparameterTuningJobs.display_name': tuningJob.display_name,
    'hyperparameterTuning.resource_name': tuningJob.resource_name,
    'hyperparameterTuning.link': job_link,
    'hyperparameterTuning.tensorboard': board_link
})

Complete the experiment run:

In [ ]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Need to add the `hyperparameterTuning` job information to each run of the experiment:

In [ ]:
for trial in tuningJob.trials:
    expRun = aiplatform.ExperimentRun(run_name = f'{RUN_NAME}-{trial.id}', experiment = EXPERIMENT_NAME)
    expRun.log_params({
        'hyperparameterTuningJobs.display_name': tuningJob.display_name,
        'hyperparameterTuning.resource_name': tuningJob.resource_name,
        'hyperparameterTuning.link': job_link,
        'hyperparameterTuning.tensorboard': board_link
    })
    expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Retrieve the experiment:

In [ ]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [ ]:
exp.get_data_frame()

Review the Experiments TensorBoard to compare runs:

In [ ]:
print(f"The Experiment TensorBoard Link:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{exp.name}")

### Review Experiment and Run in Console

In [ ]:
print(f'Review The Experiment in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}?project={PROJECT_ID}')

### Compare This Run Using Experiments

Get a list of all experiments in this project:

In [ ]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [ ]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Combine the runs from all experiments in SERIES into a single dataframe:

In [ ]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

Create ranks for models within experiment and across the entire SERIES:

In [ ]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks = ranks[~ranks['param.model.display_name'].isnull()]
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_auprc')
ranks

In [ ]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

In [ ]:
print(f"The current model is ranked {current_rank['experiment_rank'].iloc[0]} within this experiment and {current_rank['series_rank'].iloc[0]} across this series.")

### Create/Retrieve The Endpoint For This Series

In [ ]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

In [ ]:
endpoint.display_name

In [ ]:
endpoint.traffic_split

In [ ]:
deployed_models = endpoint.list_models()
#deployed_models

### Should This Model Be Deployed?
Is it better than the model already deployed on the endpoint?

In [ ]:
deploy = False
if deployed_models:
    for deployed_model in deployed_models:
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'The current model/version is already deployed.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'The current model is ranked better ({model_rank}) than a currently deployed model ({deployed_rank}).')
            break
    if deploy == False: print(f'The current model is ranked worse ({model_rank}) than a currently deployed model ({deployed_rank})')
else: 
    deploy = True
    print('No models currently deployed.')

### Deploy Model To Endpoint

In [ ]:
if deploy:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Not deploying - current model is worse ({model_rank}) than the currently deployed model ({deployed_rank})')

### Remove Deployed Models without Traffic

In [ ]:
for deployed_model in endpoint.list_models():
    if deployed_model.id in endpoint.traffic_split:
        print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

In [ ]:
endpoint.traffic_split

In [ ]:
#endpoint.list_models()

---
## Prediction

See many more details on requesting predictions in the [04Tools - Prediction](./04Tools%20-%20Prediction.ipynb) notebook.

### Prepare a record for prediction: instance and parameters lists

In [ ]:
n = 10
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({VAR_TARGET}, {VAR_OMIT}, splits)
        FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}
        WHERE splits='TEST'
        LIMIT {n}
        """
).to_dataframe()

In [ ]:
pred

In [ ]:
newobs = pred.to_dict(orient = 'records')
#newobs[0]

In [ ]:
newobs[0]

In [ ]:
#instances = [json_format.ParseDict(newobs[0], Value())]

### Get Predictions: Python Client

In [ ]:
prediction = endpoint.predict(instances = newobs[0:1])
prediction

In [ ]:
prediction = endpoint.predict(instances = newobs)
prediction

In [ ]:
prediction.predictions[0]

In [ ]:
np.argmax(prediction.predictions[0])

### Get Predictions: REST

In [ ]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": newobs[0:1]}))

In [ ]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

### Get Predictions: gcloud (CLI)

In [ ]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

---
## Remove Resources
see notebook "99 - Cleanup"